# Illustration of mix-SQP solver applied to a small data set, and a large one

## Analysis setup

We begin by loading the Distributions and LowRankApprox Julia packages, as well as some function definitions used in the code chunks below.

In [1]:
using Distributions
using LowRankApprox
include("../code/julia/datasim.jl");
include("../code/julia/likelihood.jl");
include("../code/julia/mixSQP.jl");

Next, initialize the sequence of pseudorandom numbers.

In [2]:
srand(1);

## Generate a small data set

Let's start with a smaller example with 50,000 samples.

In [3]:
x = normtmixdatasim(round(Int,5e4));

## Compute the likelihood matrix

Compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals, with $k = 20$. Note that the rows of the likelihood matrix are normalized by default.

In [4]:
sd = autoselectmixsd(x,nv = 20);
L  = normlikmatrix(x,sd = sd);
size(L)

(50000, 20)

## Fit mixture model using SQP algorithm 

First we run the mix-SQP algorithm once to precompile the function.

In [5]:
out = mixSQP(L,verbose = false);

Observe that only a small number of iterations is needed to converge to the solution of the constrained optimization problem.

In [9]:
k   = size(L,2);
x0  = ones(k)/k;
out = mixSQP(L,x = x0,lowrank = "none",eps = 1e-6);

Running SQP algorithm with the following settings:
- 50000 x 20 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- Exact derivative computation (partial QR not used).
iter      objective -min(g+1) #nnz #qp #ls
   1 3.03732706e+04 +6.30e-01   20   0   0
   2 3.03578835e+04 +6.29e-01   20  30  10
   3 3.03425042e+04 +6.28e-01   20  30  10
   4 3.03271328e+04 +6.28e-01   20  30  10
   5 3.03117692e+04 +6.27e-01   20  30  10
   6 3.02964134e+04 +6.27e-01   20  30  10
   7 3.02810655e+04 +6.26e-01   20  30  10
   8 3.02657255e+04 +6.25e-01   20  30  10
   9 3.02503933e+04 +6.25e-01   20  30  10
  10 3.02350689e+04 +6.24e-01   20  30  10
  11 3.02197524e+04 +6.24e-01   20  30  10
  12 3.02044438e+04 +6.23e-01   20  30  10
  13 3.01891430e+04 +6.22e-01   20  30  10
  14 3.01738501e+04 +6.22e-01   20  30  10
  15 3.01585650e+04 +6.21e-01   20  30  10
  16 3.01432878e+04 +6.21e-01   20  30  10
  17 3.01280185e+04 +6.20e-01   20  30  10
  18 3.01127570e+04 +6.19e

## Generate a larger data set

Next, let's see what happens when we use the SQP algorithm to fit a mixture model to a much larger data set.

In [ ]:
srand(3);
x = normtmixdatasim(round(Int,1e6));

## Compute the likelihood matrix

As before, we compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals. This time, we use a finer grid of $k = 100$ normal densities.

In [ ]:
sd = autoselectmixsd(x,nv = 100);
L  = normlikmatrix(x,sd = sd);
size(L)

## Fit mixture model using SQP algorithm 

Even on this much larger data set, only a small number of iterations is needed to compute the solution.

In [ ]:
k   = size(L,2);
x0  = ones(k)/k;
out = mixSQP(L,x = x0,lowrank = "none");

## Session information

The section gives information about the computing environment used to generate the results contained in this
notebook, including the version of Julia, and the versions of the Julia packages used here. 

In [ ]:
Pkg.status("Distributions");
Pkg.status("LowRankApprox");
versioninfo();